# dsPCA demo
This demo will apply dsPCA to an example neural population activity recorded from mice performing value-based decision making (Hattori et al., Cell, 2019). We will decompose the population activity into demixed 3 targeted value-related subspaces and a subspase which is free of the targeted 3 value-related signals.

#### Load modules

In [4]:
from dspca.dspca import dsPCA
from sklearn.decomposition import PCA
import numpy as np

ModuleNotFoundError: No module named 'dspca.dspca'; 'dspca' is not a package

#### Load data

In [2]:
with np.load(r'data/data.npz') as data:
    dQ = data['dQ']  # Target
    Qch = data['Qch']  # Target
    sQ = data['sQ']  # Target
    activity = data['activity']  # Neural population activity (Trial X Time X Cell)


NameError: name 'np' is not defined

#### Set parameters

In [ ]:
targets = np.vstack((dQ, Qch, sQ)).T    # Target task-related variables
time_range = np.arange(10, 15)  # Time range used to identify subspaces
activity_mean = np.mean(activity[:, time_range, :], axis=1)  # Temporally averaged neural population activity (Trial X Cell)

#### Run dsPCA

In [ ]:
projection_target_subspace, projection_targetfree_subspace, ax_targets, ax_targetfree, \
target_subspace_signal, targetfree_subspace_signal, target_subspace_var, targetfree_subspace_var, total_var, dot_target_ax\
    = dsPCA(data=activity_mean, targets=targets)

projection_target_subspace
- Projections of population activity to demixed target signal axes ([Trials] x [target dimensions]). The dimension type is in the order of dQ, Qch, sQ.

projection_targetfree_subspace
- Projections of population activity to the axes of target-free subspace ([Trials] x [target-free dimensions]). The dimensions are ordered according to the amount of explained activity variance.

ax_targets
- Axis vectors for the target signal subspace.

ax_targetfree
- Axis vectors for the target-free signal subspace.

target_subspace_signal
- Pearson correlation coefficient between the activity along each target axis and the targeted task-related variables. The 1st dimension indicates target axis type, the 2nd dimension indicates target variables, the 3rd dimension specifies the correlation coefficient (0) or the p-value (1).

targetfree_subspace_signal
- Pearson correlation coefficient between the activity along each target-free axis and the targeted task-related variables. The 1st dimension indicates target-free axes, the 2nd dimension indicates target variables, the 3rd dimension specifies the correlation coefficient (0) or the p-value (1).

target_subspace_var
- Activity variance along each taraget axis

targetfree_subspace_var
- Activity variance along each taraget-free axis

total_var
- Total activity variance of the original input data (activity_mean)

dot_target_ax
- Matrix with dot products between pairs of target axis vectors.

#### Project original Trial X Time X Cell tensor to a target subspace and a target-free subspace

In [ ]:
projection_target_subspace_3d = np.matmul(activity, ax_targets)
projection_targetfree_subspace_3d = np.matmul(activity, ax_targetfree)

#### Dimensions that capture the largest within-trial temporal data variance within the target subspace and the target-free subspace

In [ ]:
pc_num = 2  # Number of dimensions for temporal axes
pca_time = PCA(n_components=pc_num)
projection_target_subspace_temporal = pca_time.fit_transform(np.mean(projection_target_subspace_3d, axis=0)).shape
projection_targetfree_subspace_temporal = pca_time.fit_transform(np.mean(projection_targetfree_subspace_3d, axis=0)).shape
